## 情感分布考察

In [114]:
import os
import pandas as pd
import numpy as np
import paddlehub as hub
import imageio

In [65]:
df = pd.read_csv("xue_distance_final.csv")

In [11]:
df = df['content']

In [12]:
df

0                                              要黑胶的私我 免费七天
1                                                     cpdd
2                                               桥牌的谎言是什么意思
3                                   “你永远不知道 我假装不在乎的样子有多难受”
4        在这么冷的天?想抽根电子?烟可瑞克没有电可是雪豹已失联摘不下我虚伪的假面几句胡言被奉为圣箴尝...
                               ...                        
85260                                             我在广东没看过雪
85261                                                   第一
85262                                      我愿把胸膛解剖 公开脆弱的结构
85263                                               啊啊啊啊啊啊
85264                                                  第一！
Name: content, Length: 85265, dtype: object

In [13]:
# 加载模型
senta = hub.Module(name="senta_bilstm")

Download https://bj.bcebos.com/paddlehub/paddlehub_dev/senta_bilstm_1.2.0.tar.gz
[##################################################] 100.00%
Decompress C:\Users\Adydio\.paddlehub\tmp\tmpyo6njcds\senta_bilstm_1.2.0.tar.gz
[##################################################] 100.00%


[2023-05-11 18:17:08,216] [    INFO] - Successfully installed senta_bilstm-1.2.0
[2023-05-11 18:17:08,386] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [62]:
test_text = df.tolist()
for i in range(len(test_text)):
    test_text[i] = str(test_text[i])

In [63]:
results = senta.sentiment_classify(texts=test_text, use_gpu=False, batch_size=1)

In [66]:
results_df = pd.DataFrame(results)

In [109]:
df2 = pd.concat([df,results_df],axis=1)

In [110]:
df2["new_sentiment_label"] = df2["positive_probs"].map(lambda x: 1 if x>=0.5 else 0)
df2["p"] = df2["positive_probs"].map(lambda x: 0.5 if x>=0 else 0)

In [111]:
df2 = df2[['userid','timeStr','content','new_sentiment_label','p']]

In [112]:
df2.head()

,userid,timeStr,content,new_sentiment_label,p
0,121155230,2023/5/9,要黑胶的私我 免费七天,1,0.5
1,1521749832,2023/5/9,cpdd,1,0.5
2,274740275,2023/5/9,桥牌的谎言是什么意思,0,0.5
3,4004418265,2023/5/9,“你永远不知道 我假装不在乎的样子有多难受”,0,0.5
4,560208846,2023/5/9,在这么冷的天?想抽根电子?烟可瑞克没有电可是雪豹已失联摘不下我虚伪的假面几句胡言被奉为圣箴尝...,0,0.5


In [113]:
df2.to_csv("xue_emotion_50.csv",encoding='utf_8_sig')